In [98]:
import re
timestamp = r"(\d{1,2}:)?\d{1,2}:\d{1,2}"
wrapped_timestamp = rf"({timestamp})|(\({timestamp}\)|(\[{timestamp}\]))"
# 0:00 - title - unrelated link
timestamp_first_multiple_seperator = rf"({wrapped_timestamp})(?P<sep>( +- +)|( +\| +)|(: +))(?P<title>.*?)(?P=sep)"
timestamp_first = rf"({wrapped_timestamp})(( +- +)|( +\| +)|(: +))(?P<title>.*)$"
timestamp_first_space_seperated = rf"({wrapped_timestamp}) +(?P<title>.*?)"
title_first = rf"(?P<title>.*)(( +- +)|( +\| +)|(: +))(?P<ts>{wrapped_timestamp})$"
# non-space seperators don't work, have to assume last space if the seperator
# hence this is to be later in order of formats tried
title_first_space_seperated = rf"(?P<title>.*) +(?P<ts>{wrapped_timestamp})$" 
p = re.compile(timestamp_first)
# todo: need to strip for /?
p.match("00:00 | Neopolitin - Guustavv | random").group("title")

'Neopolitin - Guustavv | random'

In [122]:
def parse_chapter_information(line):
    timestamp = r"(\d{1,2}:)?\d{1,2}:\d{1,2}"
    wrapped_timestamp = rf"({timestamp})|(\({timestamp}\)|(\[{timestamp}\]))"
    valid_formats = [
        # 0:00 - title - unrelated link
        re.compile(rf"(?P<wrapped_ts>{wrapped_timestamp})(?P<sep>( +- +)|( +\| +)|(: +))(?P<title>.*?)(?P=sep)"),
        re.compile(rf"(?P<wrapped_ts>{wrapped_timestamp})(( +- +)|( +\| +)|(: +))(?P<title>.*)$"),
        # non-space seperators don't work, have to assume last space if the seperator
        # hence this is to be later in order of formats tried
        re.compile(rf"(?P<wrapped_ts>{wrapped_timestamp}) +(?P<title>.*?)"),
        re.compile(rf"(?P<title>.*)(( +- +)|( +\| +)|(: +))(?P<wrapped_ts>{wrapped_timestamp})$"),
        re.compile(rf"(?P<title>.*) +(?P<wrapped_ts>{wrapped_timestamp})$"),
    ]
    for format_ in valid_formats:
        match = format_.match(line)
        if match:
            wrapped_ts, title = match.group("wrapped_ts"), match.group("title")
            ts_format = re.compile(timestamp)
            ts = ts_format.search(wrapped_ts).group(0)
            return ts, match.group("title")
    return None

def parse_chapters_from_description(description):
    chapters = []
    for line in description.split("\n"):
        chapter = parse_chapter_information(line)
        if chapter:
            # youtube rules say first timestamp must start at 0:00
            if len(chapter) == 0 and chapter[0] != "0:00":
                return []
            chapters.append(chapter)
    return chapters

desc = 'Tracklist:\n\n1. Memento Mori Intro (I Love You) - 0:00\n2. One Of Those Nights (Demo) - 1:05\n3. Angel Face - 2:05\n4. For Your Eyes - 3:05\n5. Heavenly Creatures (These Girls) V2 - 8:32\n6. Another One Of Me - 12:42\n7. Money Power Glory (feat. Lana Del Rey) - 14:14\n8. Heavenly Creatures V1 (Bonus) - 15:56\n\nEnjoy!\n\nMEMENTO MORI: Episode 9 - KISS LAND EDITION\n\n'
print(parse_chapters_from_description(desc))

[('0:00', '1. Memento Mori Intro (I Love You)'), ('1:05', '2. One Of Those Nights (Demo)'), ('2:05', '3. Angel Face'), ('3:05', '4. For Your Eyes'), ('8:32', '5. Heavenly Creatures (These Girls) V2'), ('12:42', '6. Another One Of Me'), ('14:14', '7. Money Power Glory (feat. Lana Del Rey)'), ('15:56', '8. Heavenly Creatures V1 (Bonus)')]


In [118]:
timestamp = r"(\d{1,2}:)?\d{1,2}:\d{1,2}"
wrapped_timestamp = rf"({timestamp})|(\({timestamp}\)|(\[{timestamp}\]))"
p = re.compile(timestamp)
p.search("(:00:00)").group(0)

'00:00'